##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Aprendizado federado para geração de texto

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/federated_learning_for_text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/federated_learning_for_text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/federated/tutorials/federated_learning_for_text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/federated/tutorials/federated_learning_for_text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

**OBSERVAÇÃO**: foi verificado que este Colab funciona com a [versão mais recente lançada](https://github.com/tensorflow/federated#compatibility) do pacote pip `tensorflow_federated`, mas o projeto TensorFlow Federated ainda está em desenvolvimento pré-lançamento e pode não funcionar no `main`.

Este tutorial aprofunda os conceitos do tutorial [Aprendizado federado para classificação de imagens](federated_learning_for_image_classification.ipynb) e demonstra diversas outras estratégias úteis de aprendizado federado.

Especificamente, carregamos um modelo do Keras já treinado e o refinamos utilizando treinamento federado em um dataset descentralizado (simulado). Isso é especialmente importante por diversos motivos. A capacidade de usar modelos serializados facilita a combinação de aprendizado federado com outras estratégias de aprendizado de máquina. Além disso, permite usar uma gama cada vez maior de modelos pré-treinados – por exemplo, raramente é necessário treinar modelos de linguagem do zero, pois diversos modelos pré-treinados estão amplamente disponíveis (confira o [TF Hub](https://www.tensorflow.org/hub)). Em vez disso, faz mais sentido começar a partir de um modelo pré-treinado e refiná-lo usando aprendizado federado, fazendo as adaptações para as características particulares dos dados descentralizados para uma aplicação específica.

Neste tutorial, vamos começar com uma RNN que gera caracteres ASCII e vamos refiná-la usando aprendizado federado. Também vamos mostrar como os pesos finais podem ser alimentados de volta ao modelo do Keras original, o que permite fazer rapidamente a avaliação e geração de texto usando ferramentas padrão.

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated

In [ ]:
import collections
import functools
import os
import time

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(0)

# Test that TFF is working:
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

## Carregue um modelo pré-treinado

Carregamos um modelo pré-treinado seguindo o tutorial [Geração de texto usando uma RNN com execução eager](https://www.tensorflow.org/tutorials/sequences/text_generation) do TensorFlow. Porém, em vez de treinarmos usando [As obras completas de Shakespeare](http://www.gutenberg.org/files/100/100-0.txt), pré-treinamos o modelo usando o texto de [Uma história em duas cidades](http://www.ibiblio.org/pub/docs/books/gutenberg/9/98/98.txt) e [Um conto de Natal](http://www.ibiblio.org/pub/docs/books/gutenberg/4/46/46.txt) de Charles Dickens.

Exceto pela expansão do vocabulário, não modificamos o tutorial original, então esse modelo inicial não é o mais avançado, mas gera previsões razoáveis e é suficiente para as finalidades deste tutorial. O modelo final foi salvo com `tf.keras.models.save_model(include_optimizer=False)`.

Vamos usar aprendizado federado para fazer os ajustes finos do modelo de Shakespeare para este tutorial usando uma versão federada dos dados fornecida pelo TFF.


### Gere as tabelas de pesquisa de vocabulário

In [ ]:
# A fixed vocabularly of ASCII chars that occur in the works of Shakespeare and Dickens:
vocab = list('dhlptx@DHLPTX $(,048cgkoswCGKOSW[_#\'/37;?bfjnrvzBFJNRVZ"&*.26:\naeimquyAEIMQUY]!%)-159\r')

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

### Carregue o modelo pré-treinado e gere algum texto

In [ ]:
def load_model(batch_size):
  urls = {
      1: 'https://storage.googleapis.com/tff-models-public/dickens_rnn.batch1.kerasmodel',
      8: 'https://storage.googleapis.com/tff-models-public/dickens_rnn.batch8.kerasmodel'}
  assert batch_size in urls, 'batch_size must be in ' + str(urls.keys())
  url = urls[batch_size]
  local_file = tf.keras.utils.get_file(os.path.basename(url), origin=url)  
  return tf.keras.models.load_model(local_file, compile=False)

In [ ]:
def generate_text(model, start_string):
  # From https://www.tensorflow.org/tutorials/sequences/text_generation
  num_generate = 200
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(
        predictions, num_samples=1)[-1, 0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
# Text generation requires a batch_size=1 model.
keras_model_batch1 = load_model(batch_size=1)
print(generate_text(keras_model_batch1, 'What of TensorFlow Federated, you ask? '))

16193984/16193984 [==============================] - 0s 0us/step
What of TensorFlow Federated, you ask? Same yee you? Have I so,
often games in a man who rode one knee over his friend, with the
stone faces of the dread prisoners, dud a tender mastery. They
are not alive is infirmed us--to ever resume


## Carregue e pré-processe os dados de Shakespeare federados

O pacote `tff.simulation.datasets` fornece diversos datasets que são divididos em "clientes", em que cada cliente corresponde a um dataset em um dispositivo específico que pode participar do aprendizado federado.

Esses datasets fornecem distribuições de dados não distribuídos identicamente que replicam em simulações os desafios de fazer treinamento com dados descentralizados reais. Parte do pré-processamento desses dados foi feita usando-se ferramentas do [Leaf project](https://arxiv.org/abs/1812.01097) ([github](https://github.com/TalwalkarLab/leaf)).

In [ ]:
train_data, test_data = tff.simulation.datasets.shakespeare.load_data()

Os datasets fornecidos por `shakespeare.load_data()` consistem de uma sequência `Tensors` string, um para cada fala de um personagem específico de uma peça de Shakespeare. As chaves de cliente consistem do nome da peça combinado com o nome do personagem. Portanto, por exemplo, `MUCH_ADO_ABOUT_NOTHING_OTHELLO` corresponde às falas do personagem Othello na peça *Muito Barulho por Nada (Much Ado About Nothing)*. Observe que, em um cenário real de aprendizado federado, os clientes nunca são identificados ou rastreados por IDs, mas, para simulações, trabalhar com datasets com chaves é muito útil.

Aqui, por exemplo, podemos dar uma olhada nos dados de Rei Lear:

In [ ]:
# Here the play is "The Tragedy of King Lear" and the character is "King".
raw_example_dataset = train_data.create_tf_dataset_for_client(
    'THE_TRAGEDY_OF_KING_LEAR_KING')
# To allow for future extensions, each entry x
# is an OrderedDict with a single key 'snippets' which contains the text.
for x in raw_example_dataset.take(2):
  print(x['snippets'])

tf.Tensor(b'', shape=(), dtype=string)
tf.Tensor(b'What?', shape=(), dtype=string)


Agora usamos as transformações de `tf.data.Dataset` a fim de preparar esses dados para treinar a RNN de caracteres carregada acima.


In [ ]:
# Input pre-processing parameters
SEQ_LENGTH = 100
BATCH_SIZE = 8
BUFFER_SIZE = 100  # For dataset shuffling

In [ ]:
# Construct a lookup table to map string chars to indexes,
# using the vocab loaded above:
table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        keys=vocab, values=tf.constant(list(range(len(vocab))),
                                       dtype=tf.int64)),
    default_value=0)


def to_ids(x):
  s = tf.reshape(x['snippets'], shape=[1])
  chars = tf.strings.bytes_split(s).values
  ids = table.lookup(chars)
  return ids


def split_input_target(chunk):
  input_text = tf.map_fn(lambda x: x[:-1], chunk)
  target_text = tf.map_fn(lambda x: x[1:], chunk)
  return (input_text, target_text)


def preprocess(dataset):
  return (
      # Map ASCII chars to int64 indexes using the vocab
      dataset.map(to_ids)
      # Split into individual chars
      .unbatch()
      # Form example sequences of SEQ_LENGTH +1
      .batch(SEQ_LENGTH + 1, drop_remainder=True)
      # Shuffle and form minibatches
      .shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
      # And finally split into (input, target) tuples,
      # each of length SEQ_LENGTH.
      .map(split_input_target))

Observe que, na formação das sequências originais e na formação dos lotes acima, usamos `drop_remainder=True` por questão de simplicidade. Portanto, todos os personagens (clientes) que não têm pelo menos `(SEQ_LENGTH + 1) * BATCH_SIZE` caracteres de texto terão datasets vazios. Uma estratégia típica para tratar isso seria preencher os lotes com um token especial e depois usar uma máscara na perda para não levar os tokens de preenchimento em consideração.

Isso complicaria um pouco o exemplo, então, para este tutorial, usamos somente lotes completos, como no [tutorial padrão](https://www.tensorflow.org/tutorials/sequences/text_generation). Entretanto, em um ambiente federado, esse problema é mais significativo, pois muitos usuários podem ter datasets pequenos.

Agora, podemos pré-processar o `raw_example_dataset` e verificar os tipos:

In [ ]:
example_dataset = preprocess(raw_example_dataset)
print(example_dataset.element_spec)

(TensorSpec(shape=(8, 100), dtype=tf.int64, name=None), TensorSpec(shape=(8, 100), dtype=tf.int64, name=None))


## Compile o modelo e teste com os dados pré-processados

Carregamos um modelo do Keras não compilado, mas, para executar `keras_model.evaluate`, precisamos compilá-lo com uma perda e métricas. Também vamos compilar em um otimizador, que será usado como o otimizador no dispositivo para o aprendizado federado.

O tutorial original não tinha exatidão em nível de caracteres (a fração das previsões em que a probabilidade mais alta foi colocada no caractere correto). Essa é uma métrica útil, então vamos adicioná-la. Porém, precisamos definir uma nova classe de métricas para isso, pois nossas previsões têm posto 3 (um vetor de logits para cada uma das `BATCH_SIZE * SEQ_LENGTH` previsões), e `SparseCategoricalAccuracy` espera somente previsões de posto 2.

In [ ]:
class FlattenedCategoricalAccuracy(tf.keras.metrics.SparseCategoricalAccuracy):

  def __init__(self, name='accuracy', dtype=tf.float32):
    super().__init__(name, dtype=dtype)

  def update_state(self, y_true, y_pred, sample_weight=None):
    y_true = tf.reshape(y_true, [-1, 1])
    y_pred = tf.reshape(y_pred, [-1, len(vocab), 1])
    return super().update_state(y_true, y_pred, sample_weight)

Agora podemos compilar um modelo e avaliá-lo para nosso `example_dataset`.

In [ ]:
BATCH_SIZE = 8  # The training and eval batch size for the rest of this tutorial.
keras_model = load_model(batch_size=BATCH_SIZE)
keras_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[FlattenedCategoricalAccuracy()])

# Confirm that loss is much lower on Shakespeare than on random data
loss, accuracy = keras_model.evaluate(example_dataset.take(5), verbose=0)
print(
    'Evaluating on an example Shakespeare character: {a:3f}'.format(a=accuracy))

# As a sanity check, we can construct some completely random data, where we expect
# the accuracy to be essentially random:
random_guessed_accuracy = 1.0 / len(vocab)
print('Expected accuracy for random guessing: {a:.3f}'.format(
    a=random_guessed_accuracy))
random_indexes = np.random.randint(
    low=0, high=len(vocab), size=1 * BATCH_SIZE * (SEQ_LENGTH + 1))
data = collections.OrderedDict(
    snippets=tf.constant(
        ''.join(np.array(vocab)[random_indexes]), shape=[1, 1]))
random_dataset = preprocess(tf.data.Dataset.from_tensor_slices(data))
loss, accuracy = keras_model.evaluate(random_dataset, steps=10, verbose=0)
print('Evaluating on completely random data: {a:.3f}'.format(a=accuracy))

16193984/16193984 [==============================] - 0s 0us/step
Evaluating on an example Shakespeare character: 0.45.000
Expected accuracy for random guessing: 0.012
Evaluating on completely random data: 0.011


## Faça os ajustes finos do modelo com aprendizado federado

O TFF serializa todas as computações do TensorFlow para que elas possam ser executadas em um ambiente que não é Python (muito embora, no momento, somente um runtime de simulação implementado no Python esteja disponível). Embora estejamos fazendo a execução no modo eager (TF 2.0), atualmente o TFF serializa as computações do TensorFlow construindo os operadores necessários dentro do contexto de uma declaração "`with tf.Graph.as_default()`". Portanto, precisamos fornecer uma função que o TFF possa usar para apresentar nosso modelo a um grafo que ele controle, o que pode ser feito da seguinte maneira:

In [ ]:
# Clone the keras_model inside `create_tff_model()`, which TFF will
# call to produce a new copy of the model inside the graph that it will 
# serialize. Note: we want to construct all the necessary objects we'll need 
# _inside_ this method.
def create_tff_model():
  # TFF uses an `input_spec` so it knows the types and shapes
  # that your model expects.
  input_spec = example_dataset.element_spec
  keras_model_clone = tf.keras.models.clone_model(keras_model)
  return tff.learning.models.from_keras_model(
      keras_model_clone,
      input_spec=input_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[FlattenedCategoricalAccuracy()])

Agora está tudo pronto para construirmos um processo iterativo de cálculo federado de médias, que usaremos para melhorar o modelo (confira detalhes sobre o algoritmo de cálculo federado de médias no artigo [Communication-Efficient Learning of Deep Networks from Decentralized Data](https://arxiv.org/abs/1602.05629) – Aprendizado de redes profundas com comunicação eficiente usando dados descentralizados).

Usamos um modelo do Keras compilado para fazer a avaliação padrão (não federada) após cada rodada do treinamento federado, o que é útil para fins de pesquisa ao fazer o aprendizado federado simulado quando há um dataset de teste padrão.

Em um ambiente de produção mais realista, essa mesma técnica pode ser usada para pegar modelos treinados com aprendizado federado e avaliá-los com um dataset de benchmark centralizado para fins de teste ou garantia de qualidade.

In [ ]:
# This command builds all the TensorFlow graphs and serializes them: 
fed_avg = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.5))

Veja abaixo o loop mais simples possível, em que executamos o cálculo federado de médias para uma rodada para um único cliente em um único lote:

In [ ]:
state = fed_avg.initialize()
result = fed_avg.next(state, [example_dataset.take(5)])
state = result.state
train_metrics = result.metrics['client_work']['train']
print('loss={l:.3f}, accuracy={a:.3f}'.format(
    l=train_metrics['loss'], a=train_metrics['accuracy']))

loss=4.399, accuracy=0.139


Agora vamos escrever um loop de treinamento e avaliação ligeiramente mais interessante.

Para que essa simulação ainda tenha uma execução relativamente rápida, treinamos nos mesmos três clientes em cada rodada, considerando somente dois minilotes para cada um.


In [ ]:
def data(client, source=train_data):
  return preprocess(source.create_tf_dataset_for_client(client)).take(5)


clients = [
    'ALL_S_WELL_THAT_ENDS_WELL_CELIA', 'MUCH_ADO_ABOUT_NOTHING_OTHELLO',
]

train_datasets = [data(client) for client in clients]

# We concatenate the test datasets for evaluation with Keras by creating a 
# Dataset of Datasets, and then identity flat mapping across all the examples.
test_dataset = tf.data.Dataset.from_tensor_slices(
    [data(client, test_data) for client in clients]).flat_map(lambda x: x)

O estado inicial do modelo gerado por `fed_avg.initialize()` é baseado nos inicializadores aleatórios para o modelo do Keras, e não nos pesos que foram carregados, já que `clone_model()` não clona os pesos. Para começar o treinamento a partir de um modelo pré-treinado, definimos os pesos do modelo no estado do servidor diretamente a partir do modelo carregado.

In [ ]:
NUM_ROUNDS = 5

# The state of the FL server, containing the model and optimization state.
state = fed_avg.initialize()

# Load our pre-trained Keras model weights into the global model state.
pre_trained_weights = tff.learning.models.ModelWeights(
    trainable=[v.numpy() for v in keras_model.trainable_weights],
    non_trainable=[v.numpy() for v in keras_model.non_trainable_weights]
)
state = fed_avg.set_model_weights(state, pre_trained_weights)


def keras_evaluate(state, round_num):
  # Take our global model weights and push them back into a Keras model to
  # use its standard `.evaluate()` method.
  keras_model = load_model(batch_size=BATCH_SIZE)
  keras_model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[FlattenedCategoricalAccuracy()])
  model_weights = fed_avg.get_model_weights(state)
  model_weights.assign_weights_to(keras_model)
  loss, accuracy = keras_model.evaluate(example_dataset, steps=2, verbose=0)
  print('\tEval: loss={l:.3f}, accuracy={a:.3f}'.format(l=loss, a=accuracy))


for round_num in range(NUM_ROUNDS):
  print('Round {r}'.format(r=round_num))
  keras_evaluate(state, round_num)
  result = fed_avg.next(state, train_datasets)
  state = result.state
  train_metrics = result.metrics['client_work']['train']
  print('\tTrain: loss={l:.3f}, accuracy={a:.3f}'.format(
      l=train_metrics['loss'], a=train_metrics['accuracy']))

print('Final evaluation')
keras_evaluate(state, NUM_ROUNDS + 1)

Round 0
	Eval: loss=3.171, accuracy=0.428
	Train: loss=4.309, accuracy=0.098
Round 1
	Eval: loss=4.188, accuracy=0.185
	Train: loss=4.037, accuracy=0.223
Round 2
	Eval: loss=3.948, accuracy=0.200
	Train: loss=3.797, accuracy=0.228
Round 3
	Eval: loss=3.826, accuracy=0.179
	Train: loss=3.662, accuracy=0.219
Round 4
	Eval: loss=3.723, accuracy=0.171
	Train: loss=3.440, accuracy=0.245
Final evaluation
	Eval: loss=3.599, accuracy=0.181


Com as alterações padrão, não fizemos treinamento suficiente para que a diferença fosse grande, mas, se você treinar por mais tempo e com mais dados de Shakespeare, deverá ver uma diferença no estilo do texto gerado ao usar o modelo atualizado:

In [ ]:
# Set our newly trained weights back in the originally created model.
keras_model_batch1.set_weights([v.numpy() for v in keras_model.weights])
# Text generation requires batch_size=1
print(generate_text(keras_model_batch1, 'What of TensorFlow Federated, you ask? '))

What of TensorFlow Federated, you ask? She will be
heard of; or whether they recovered her faltering place, that a great mark of
being so final dark and distrustner the dearer to the chin, all
staftly towards him, or trot's in foot thro


## Extensões sugeridas

Este tutorial é apenas o primeiro passo! Veja algumas ideias para estender este notebook:

- Escreva um loop de treinamento mais realista, em que você amostre clientes para fazer o treinamento aleatoriamente.
- Use "`.repeat(NUM_EPOCHS)`" nos datasets clientes para tentar diversas épocas de treinamento local (como em [McMahan et. al.](https://arxiv.org/abs/1602.05629)). Confira [Aprendizado federado para classificação de imagens](federated_learning_for_image_classification.ipynb), que faz isso.
- Altere o comando `compile()` para fazer experimentos usando diferentes algoritmos de otimização em cada cliente.
- Experimente o argumento `server_optimizer` em `build_weighted_fed_avg` para tentar diferentes algoritmos para aplicar as atualizações do modelo no servidor.